# Using Java to access SQL databases

This Notebook explains how to work with relational databases from Java. The database used here is called H2, to be found here: [H2 Home Page](https://www.h2database.com/html/main.html). H2 is only one of very many different Open Source databases available for Java. 

The Notebook uses H2 as a library inside the Java application. But it can be used as a standalone DBMS. 
    

# Getting started in Java

The first thing one has to do in a Java application when working with relational databases is to import the appropriate class files:

In [18]:
import java.sql.Connection;
import java.sql.DriverManager;
import java.sql.Statement;
import java.sql.ResultSet;
import java.sql.ResultSetMetaData;
import java.sql.SQLException;

import java.util.*;



Java uses an abstraction layer, called JDBC, to work with databases. This way, the Java program can access the same methods without worrying which database it works with. JDBC loads the correct driver for the database and translates back and forth between the Java program and the database driver.

In order to allow JDBC to find the driver, one needs to include the driver jar(s) into the classpath.


In [2]:
%classpath libs/h2-2.3.232.jar

In the next cell, three variables are set up for the handling of the access to the database:

| Variable name | Value | Example |
| ------------- | ----- | ------- |
| DB_URL        | Connection string / URL to access the Database. It is constructions like this: <br> \<PROTOCOL\>:\<DATABASE DRIVER\>:\<Where to find the db\>. | jdbc:h2:./dbs/orderdb
| USER          | User accessing the db. | sa  ( = System Administrator) |
| PASS          | Password for user, | 1234  (H2 standard password) |

In [15]:
String DB_URL = "jdbc:h2:./dbs/orderdb";
String USER   = "sa";
String PASS   = "1234";

The next cell shows the code on how to connect to the database (H2 in our case). One would use this setup only once in a program, and always use the connection established here. This is not possible in a Notebook. Therefore, all the Java code cells require this boilerplate code.

The try statement is handling our resource, in this case the database connection, for us. It makes sure that the connection gets close, no matter if the commands were run successfully or if there happens an exception.

In [4]:
try (Connection conn = DriverManager.getConnection(DB_URL, USER, PASS)) {
    
    //   <<SQL Statements to process>>
                
    System.out.println("Successfully run the SQL statements.");

} catch ( Exception ex ) {
    System.err.println("ERROR: " + ex.getMessage());
}

Successfully run the SQL statements.


The following code cell contains the helper function printFormatted. Please run it to allow its usage.

In [20]:
private String padRight(String text, int width) {
    return String.format("%-" + width + "s", text);
}

public void printFormatted(ResultSet rs) throws SQLException {
    ResultSetMetaData meta = rs.getMetaData();
    int columnCount = meta.getColumnCount();

    // Schritt 1: Spaltennamen und Daten zwischenspeichern
    List<String[]> rows = new ArrayList<>();
    int[] maxWidths = new int[columnCount];

    // Spaltennamen in erste Zeile speichern und Breiten ermitteln
    String[] header = new String[columnCount];
    for (int i = 1; i <= columnCount; i++) {
        header[i - 1] = meta.getColumnLabel(i);
        maxWidths[i - 1] = header[i - 1].length();
    }

    // Daten durchlaufen und Breiten ermitteln
    while (rs.next()) {
        String[] row = new String[columnCount];
        for (int i = 1; i <= columnCount; i++) {
            String value = rs.getString(i);
            if (value == null) value = "NULL";
            row[i - 1] = value;
            maxWidths[i - 1] = Math.max(maxWidths[i - 1], value.length());
        }
        rows.add(row);
    }

    // Schritt 2: Ausgabe
    // Header ausgeben
    for (int i = 0; i < columnCount; i++) {
        System.out.print(padRight(header[i], maxWidths[i]) + " | ");
    }
    System.out.println();

    // Trennlinie
    for (int i = 0; i < columnCount; i++) {
        System.out.print("-".repeat(maxWidths[i]) + "-+-");
    }
    System.out.println();

    // Datenzeilen ausgeben
    for (String[] row : rows) {
        for (int i = 0; i < columnCount; i++) {
            System.out.print(padRight(row[i], maxWidths[i]) + " | ");
        }
        System.out.println();
    }
}



In order to execute a SQL Statement one needs to use the Statement class. Whatever is given back from the database running the statement is then received as a Resultset. The next() call runs through all the records in the resultset. One accesses each attribute of a record by name. There is the need to know the data type of the attribute accessed.

(In case of the printFormatted method, the only data type used is String, enforced by the rs.getString call.)

In [21]:
void get_10_C_Products(Connection conn) throws java.sql.SQLException {
    final String QUERY = "SELECT * FROM Products where ProductName like 'C%' Limit 10;";
    try(Statement stmt = conn.createStatement()) {
        ResultSet rs = stmt.executeQuery(QUERY);
        printFormatted(rs);
    }
    return;
}

try (Connection conn = DriverManager.getConnection(DB_URL, USER, PASS)) {
    
    get_10_C_Products(conn);
                
    System.out.println("Successfully run the SQL statements.");

} catch ( Exception ex ) {
    System.err.println("ERROR: " + ex.getMessage());
}

ID | PRODUCTNAME                  | SUPPLIERID | UNITPRICE | PACKAGE            | ISDISCONTINUED | 
---+------------------------------+------------+-----------+--------------------+----------------+-
1  | Chai                         | 1          | 18.00     | 10 boxes x 20 bags | FALSE          | 
2  | Chang                        | 1          | 19.00     | 24 - 12 oz bottles | FALSE          | 
4  | Chef Anton's Cajun Seasoning | 2          | 22.00     | 48 - 6 oz jars     | FALSE          | 
5  | Chef Anton's Gumbo Mix       | 2          | 21.35     | 36 boxes           | TRUE           | 
18 | Carnarvon Tigers             | 7          | 62.50     | 16 kg pkg.         | FALSE          | 
38 | Côte de Blaye                | 18         | 263.50    | 12 - 75 cl bottles | FALSE          | 
39 | Chartreuse verte             | 18         | 18.00     | 750 cc per bottle  | FALSE          | 
48 | Chocolade                    | 22         | 12.75     | 10 pkgs.           | FALSE          | 


# Avoiding SQL-Injections

## Insecure construction of SQL Statement

Let's say one wants to select a Customer with a specific last name from the customers table.

> String customerName = "' OR '1'='1"; // <-- this is the attack input <br>
String sql = "SELECT * FROM Customers WHERE lastname = '" + customerName + "'";

Here, the final Strings looks like this:

> SELECT * FROM Customers WHERE lastname = '' OR '1'='1';

This matches all records! 

Imagine this:

> String customerName = "'; Drop Table Customers; Commit;"; // <-- this is the attack input <br>

**Good bye Customers table!**

## The safe way

In order to prevent this, one uses PreparedStatements:

> String customerName = "' OR '1'='1"; // same input <br>
String sql = "SELECT * FROM Customers WHERE customerName = ?"; <br> <br>
PreparedStatement stmt = connection.prepareStatement(sql);<br>
stmt.setString(1, customerName);<br> <br>
ResultSet rs = stmt.executeQuery();

The SQL statement in this case looks similar to this:  

> SELECT * FROM Customers WHERE lastname = '\' OR \'1\'=\'1';

The Preparedstatement masks all the apostrophes and double quotes, so that the parameter passed into the Preparedstatement is taken as a single string. No chances to inject other statements.


# ORM

ORM stands for Object-Relational Mapping.

It is a programming technique that allows developers to interact with a relational database using objects instead of writing raw SQL.

An ORM maps:

* Database tables → Classes

* Table rows → Objects

* Table columns → Object fields

<img src="pics/ORM_1.png">
<img src="pics/ORM_2.png">
<img src="pics/ORM_3.png">
<img src="pics/ORM_4.png">
<img src="pics/ORM_5.png">